In [6]:
# criar o navegador
import os
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By

servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico)

def importando_site_e_planilha():
    # abrir a página index (entrar no site da busca jurídica)
    caminho = os.getcwd()
    arquivo = caminho + r"\index.html"

    # importar a base de dados
    tabela = pd.read_excel("Processos.xlsx")
    tabela = tabela.applymap(str)
    return arquivo, tabela

def mudar_pagina(linha):
    # mudar para nova aba
    aba_original = navegador.window_handles[0]
    indice = 1 + linha
    nova_aba = navegador.window_handles[indice]

    navegador.switch_to.window(nova_aba)

def preenchendo_forms(tabela, linha):
    # preencher dados de acordo com a planilha e depois clicar no botão
    navegador.find_element(By.ID, 'nome').send_keys(tabela.loc[linha, "Nome"])
    navegador.find_element(By.ID, 'advogado').send_keys(tabela.loc[linha, "Advogado"])
    navegador.find_element(By.ID, 'numero').send_keys(tabela.loc[linha, "Processo"])

    time.sleep(0.5)

    navegador.find_element(By.CLASS_NAME, 'registerbtn').click()

    time.sleep(0.5)

def alerta_opcoes(tabela, linha):
    # aceitar ou não alerta e registrar na planilha
    alerta = navegador.switch_to.alert
    alerta.accept()

    while True:
        try:
            alerta = navegador.switch_to.alert
            break
        except:
            time.sleep(1)
    texto_alerta = alerta.text

    if "Processo encontrado com sucesso" in texto_alerta:
        tabela.loc[linha, "Status"] = "Encontrado"
        alerta.accept()
    else:
        tabela.loc[linha, "Status"] = "Não Encontrado"
        alerta.accept()
    
    time.sleep(1)

def fechar_navegador_salvar_planilha_(tabela):
    navegador.quit()

    tabela.to_excel("Processos Atualizado.xlsx")

def escolher_cidade_preencher_dados(arquivo, tabela):
    # preencher dados utilizando as funções

    for linha in tabela.index:
        navegador.get(arquivo)

        time.sleep(1)

        botao = navegador.find_element(By.CLASS_NAME, 'dropdown-menu')
        ActionChains(navegador).move_to_element(botao).perform()

        time.sleep(0.5)

        cidade = tabela.loc[linha, "Cidade"]

        # selecionando o cidade a partir do link
        navegador.find_element(By.PARTIAL_LINK_TEXT, cidade).click()

        time.sleep(1)

        mudar_pagina(linha)

        preenchendo_forms(tabela, linha)

        alerta_opcoes(tabela, linha)

    fechar_navegador_salvar_planilha_(tabela)

arquivo, tabela = importando_site_e_planilha()
escolher_cidade_preencher_dados(arquivo, tabela)

C:\Users\dark860\AppData\Local\Temp\ipykernel_11408\2907745600.py:22: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  tabela = tabela.applymap(str)
